In [40]:
import torch
import torchvision.models as models 
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt 
  


In [41]:
Arabian_camel= Image.open('imageNet_Images/Arabian_camel.JPEG')
bluetick= Image.open('imageNet_Images/bluetick.JPEG')
cougar= Image.open('imageNet_Images/cougar.JPEG')
lion= Image.open('imageNet_Images/lion.JPEG')
ostrich= Image.open('imageNet_Images/ostrich.JPEG')
peacock= Image.open('imageNet_Images/peacock.JPEG')
tiger_shark= Image.open('imageNet_Images/tiger_shark.JPEG')
vine_snake= Image.open('imageNet_Images/vine_snake.JPEG')


In [42]:
images = [Arabian_camel, bluetick, cougar,lion,ostrich, peacock, tiger_shark, vine_snake]
images = [image.convert('RGB') for image in images] #change images to RGB 


In [43]:
import torchvision.transforms as transforms

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

processed_images = [preprocess(image) for image in images]


In [44]:
import torch
import torchvision.models as models
import torch.nn.functional as F

class MobileNetV2WithFeatures(models.MobileNetV2):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, x):
        
        x = self.features(x)

        feature_maps = x

        
        x = F.adaptive_avg_pool2d(x, (1, 1))  
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x, feature_maps


In [37]:
model = MobileNetV2WithFeatures()
model.eval()  


MobileNetV2WithFeatures(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNor

In [45]:
for input_tensor in processed_images:
    input_tensor = input_tensor.unsqueeze(0)  

    
    output, feature_maps = model(input_tensor)


    fc_layer = model.classifier[1]
    fc_weights = fc_layer.weight.data

    
    class_idx = torch.argmax(output)

    
    class_weights = fc_weights[class_idx]

    
    cam = torch.zeros_like(feature_maps[0])
    for i, w in enumerate(class_weights):
        cam += w * feature_maps[0, i, :, :]

    
    cam = F.relu(cam)

    cam = cam - cam.min()
    cam = cam / cam.max()

    


In [39]:
from PIL import Image

image_paths = [ 'bluetick.JPEG', 'cougar.JPEG', 'lion.JPEG', 'ostrich.JPEG', 'peacock.JPEG', 'tiger_shark.JPEG', 'vine_snake.JPEG']  
original_images = [Image.open(path) for path in image_paths]


FileNotFoundError: [Errno 2] No such file or directory: 'bluetick.JPEG'